## 생성 RNN의 간단한 역사

여기서는 RNN기법으로 시퀀스 데이터를 생성하는 방법을 보겠다. 텍스트 생성을 예로 들지만 동일한 기법으로 어떤 종류의 시퀀스 데이터도 생성할 수 있다. 음표의 시퀀스에 적용하여 새로운 음악을 만들거나 연속된 붓질 시퀀스에 적용하여 한 획 한 획 그림을 그릴 수 있다.

예술 뿐만아니라, 음성 합성과 챗봇의 대화 기능에도 성공적으로 적용되었다. 구글이 2016년에 공개한 스마트 답장도 빕스한 기술을 사용한다. 짧은 문장을 자동으로 생성하여 이메일이나 문자 메시지로 답장을 보낼 수 있다.

## 시퀀스 데이터를 어떻게 생성할까?

딥러닝에서 시퀀스 데이터를 생성하는 일반적인 방법은 이전 토큰을 입력으로 사용하여 시퀀스의 다음 1개 또는 몇 개의 토큰을 예측하는 것이다. 예를 들어 'the cat is on ma'란 입력이 주어지면 다음 글자인 타깃 't'를 예측하도록 네트워크를 훈련한다. 텍스트 데이터를 다룰 때 토큰은 보통 단어 또는 글자이다. 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크를 언어 모델이라고 부른다. 언어 모델은 언어의 통계적 구조인 잠재 공간을 탄색한다.

초키 텍스트 문자열을 주입하고(conditioning data) 새로운 글자나 단어를 생성한다. 생성된 출력은 다시 입력 데이터로 추가된다. 이 과정을 여러 번 반복한다. 이런 반복을 통해 모델이 훈련의 비슷한 시퀀스를 만든다. 텍스트 말뭉치(corpus)에서 N개의 글자로 이루어진 문자열을 추출하여 주입하고 N + 1번째 글자를 예측하도록 훈련한다. 모델의 출력은 출력 가능한 모든 글자에 해당하는 소프트 맥스 값이다. 즉 다음 글자의 확률 분포이다. 이 LSSTM을 글자 수준의 신경망 언어 모델(character-level neural language model)이라고 부른다.

## 샘플링 전략의 중요성

텍스트를 생성할 때 다음 글자를 선택하는 방법은 매우 중요하다. 가장 높은 확률을 가진 글자를 선택하는 탐욕적 샘플링(greedy sampling)과 다음 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 확률적 샘플링(stochastic sampling)이 있다. 확률적 샘플링을 주로 사용하며, 이 방식을 사용할 겨웅 'e'가 다음 글자가 될 확률이 0.3이라면, 모델이 30% 정도로 이 글자를 선택한다.

모델의 소프트맥스 출력은 확률적 샘플링에 사용하기 좋다. 훈련 데이터에는 없지만 실제 같은 새로운 단어를 만들어 재미있고 창의적으로 보이는 문장을 생성한다. 

이때 이 확률 분포의 엔트로피를 조절하는 것이 중요하다. 작은 엔트로피는 예상 가능한 구조를 가진 시퀀스를 생성한다. 즉 더 실제처럼 보인다. 반면 높은 엔트로프는 놀랍고 창의적인 시퀀스를 만든다. 생성 모델에서 샘플링을 할 때 생성 과정에 무작위성의 양을 바꾸어 시도해 보는 것이 좋다. 흥미는 매우 주관적이므로 최적의 엔트로피 값을 미리 알 수 없기 때문이다. 얼마나 흥미로운 데이터를 생성할 것인지는 결국 사람이 판단해야한다.

샘플링 과정에서 확률의 양을 조절하기 위해 소프트맥스 온도(softmax temperature)라는 파라미터를 사용한다. 이 파라미터는 샘플링에 사용되는 확률 분포의 엔트로피를 나탙낸다. 얼마나 놀라운 또는 예상되는 글자를 선택할지 결정한다. temperature 값이 주어지면 다음과 같이 가중치를 적용하여 (모델의 소프트맥스 출력인) 원본 확률 본포에서 새로운 확률 분포를 계산한다.

## 데이터 전처리

니체의 글을 다운 받고 소문자로 처리한다.

In [1]:
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print("size of corpus : ", len(text))

size of corpus :  600893


In [3]:
type(text)

str

maxlen 길이를 가진 시퀀스를 중복하여 추출한다. 추출된 시퀀스를 one-hot 인코들으로 변환하고 크기가 (sequences, maxlen, unique_characteers)인 3D 넘파이 배열 x로 합친다. 동시에 훈련 샘플에 상응하는 타깃을 담은 배열 y를 준비한다. 타깃은 추출된 시퀀스 다음에 오는 one-hat 인코딩된 글자이다.

In [4]:
maxlen = 60
step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i+maxlen])
    
print ('num of sequence : ', len(sentences))
    
chars = sorted(list(set(text)))
print('unique char : ', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
    
print("vectorization...")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]]=1
    y[i, char_indices[next_chars[i]]] = 1

num of sequence :  200278
unique char :  58
vectorization...


In [5]:
from keras import layers

In [6]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

타깃이 ont-hat 인코딩되어 있기 때문에 모델을 훈련하기 위해 categorial_crossentropy를 사용한다

In [7]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 언어 모델 훈련과 샘플링

훈련된 모델과 seed로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있다.

1. 지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑기
2. 특정 온도로 이 확률 분포의 가중치 조정
3. 가중치가 조정된 분포에서 무작위로 새로운 글자 샘플링
4. 새로운 글자를 생성된 텍스트의 끝에 추가

다음은 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추합하는 셈플링 함수이다.

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성한다. 에포크마다 학습이 끝난 후 여러 가지 온도를 사용하여 텍스트를 생성한다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트가 어떻게 진화하는지 볼 수 있다. 온도가 샘플링 전략에 미치는 영향도 보여준다.

In [9]:
import random
import sys

In [10]:
random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
    print("epoch", epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    seed_text = text[start_index: start_index + maxlen]
    print('--- seed text : "' + seed_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------temperature : ', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 158s 790us/step - loss: 1.9562
--- seed text : "the slowly ascending ranks and classes, in which,
through fo"
------temperature :  0.2
the slowly ascending ranks and classes, in which,
through for the conder and and in the command the conderstand and in the some in the conderstand and the command and in all the conderstand in the conderstand of the conders and the consequence of the consequent the conder and the order the conderstand and the whole here an all the consequent the command and the conders and order the conderstand it is also the condersual the command and the condequent the c
------temperature :  0.5
the slowly ascending ranks and classes, in which,
through for which when the caster and therefort the reasted become and the when that it is a full of the here a which when the calse who sufferent be is not of one it wishesy it is extrust fas it with a  nother of has the belies and the like it is in a cause as 

even certoing, in mysupreque-cousifils; many plane in relireh of his will--and fore), as given irrough
already
conpleain causuelly and haage doavily alporative wes rapseol much look too su fait, "e repire
being,"e
for plevinch
and
evonity upequarngwiy himself (dom to read by slaw! hitherto refuedek" in the ne? thousean and stappea. demongt
of eesefr
epoch 5
Epoch 1/1
200278/200278 [==============================] - 156s 779us/step - loss: 1.4578
--- seed text : "the slowly ascending ranks and classes, in which,
through fo"
------temperature :  0.2
the slowly ascending ranks and classes, in which,
through for the sense of the pressional properson of the sense of the distince and the constraited by the sense of the sense of the assumed of the sense of the most constraite and something and the constraite and art of the present the sense of the sense of the distingual and the sense of the sense of the sense of the present the sense of the present of the constraite and so that the most the 

KeyboardInterrupt: 

여기서 볼 수 있듯이 낮은 온도는 반복적이고 예상되는 텍스트를 만든다. 하지만 국보적인 구조는 실제와 매우 같다. 특히 모든 단어가 실제 영어 단어이다. 높은 온도에서 생성된 텍스트는 아주 흥미롭고 창의적이기도하다. 이따금 꽤 그럴싸하게 보이는 완전히 새로운 단어를 창조한다. 높은 온도에서는 국부적인 구조가 무너지기 시작한다. 대부분의 단어가 어느 정도 무작위한 문자열로 보인다. 확실히 이 네트워크에서는 텍스트 생성에 가장 좋은 온도는 0.5이다. 항상 다양한 샘플링 전략으로 실험해 보아야한다. 학습된 구주와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있다.

더 많은 데이터에서 크고 깊은 모델을 훈련하면 이것보다 훨씬 논리적이고 실제와 같은 텍스트 샘플을 생성할 수 있다. 이 차이를 검증하기 위해 다음과 같은 사고 실험을 해봐라. 컴퓨터는 대부분의 디지털 통신에서 하는 것처럼 사람의 언어가 의사소통을 압축하는 데 더 뛰어나다면 어떨까? 언어 의미가 줄지 않지만 고유한 통계 구조가 사라질 것이다. 이는 방금과 같은 언어 모델을 학습하는 것을 불가능하게 만든다.

## 정리

- 이전의 토큰이 주어지면 다음 토큰들을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있다.
- 텍스트의 경우 이런 모델을 언어 모델이라고 부른다. 단어 또는 글자 단위 모두 가능하다.
- 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 한다.
- 이를 위해 소프트맥스 온도 개념을 사용한다. 항상 다양한 온도를 실험해서 적절한 값을 찾아야 한다.